# Data collection and processing

## Colllecting data from AEMET

### Importo las librerias necesarias para el proyecto

In [9]:
from datetime import datetime, timedelta
import time
import json
from collections import namedtuple
import pandas as pd
import matplotlib.pyplot as plt

### Extraigo los datos de 3 años, desde el 01-01-2018 hasta el 31-12-2020 y lo almaceno en un JSON

#### Para ello, me registro en la AEMET para obtener un API key y extraigo los datos de la climatologia diaria de estos años de la estacion meteorológica situada en Ciudad Universitaria

#### Vamos a usar los siguientes datos proporcionados por la AEMET: Fecha, temperaturaMedia, precipitaciones, temperaturaMinima y temperaturaMaxima

In [27]:
ruta_archivo_json = 'datos.json'

with open(ruta_archivo_json) as archivo_json:
    datos_json = json.load(archivo_json)
    
df = pd.DataFrame(datos_json)

#Quito las columnas que no voy a necesitar
df = df.drop(['indicativo','nombre','provincia','altitud','horatmin','horatmax','dir','velmedia','racha','horaracha'],axis=1)

#### Debido a que no tengo demasiado conocimiento sobre climatologia, hare como en el tutorial proporcionado y usare informacion de los 3 dias anteriores

#### Para cada dia (fila), y cada medida (columna) voy a buscar el valor de esa medida N dias anteriores.

#### Es decir, creare una nueva columna para esa medida que represente su valor N dias antes

In [34]:
tmp = df.head(10)

In [37]:
# ALGORITMO PARA 1 DIA ANTES Y SOLO PARA UN PEQUEÑO DATAFRAME DE 10 FILAS
# 1 dia antes
N = 1

# target measurement of mean temperature
feature = 'tmed'

# total number of rows
rows = tmp.shape[0]

# a list representing Nth prior measurements of feature
# notice that the front of the list needs to be padded with N
# None values to maintain the constistent rows length for each N
nth_prior_measurements = [None]*N + [tmp[feature][i-N] for i in range(N, rows)]

# make a new column name of feature_N and add to DataFrame
col_name = "{}_{}".format(feature, N)
tmp[col_name] = nth_prior_measurements
tmp

<ipython-input-37-04123b954dba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[col_name] = nth_prior_measurements


,fecha,tmed,prec,tmin,tmax,tmed_1
0,2018-01-01,"6,9","0,0","1,7","12,1",None
1,2018-01-02,"10,4","0,0","6,3","14,6","6,9"
2,2018-01-03,"11,2","0,0","6,8","15,5","10,4"
3,2018-01-04,"11,8","0,0","7,9","15,6","11,2"
4,2018-01-05,"7,8","12,9","5,2","10,4","11,8"
5,2018-01-06,"3,3","21,6","1,3","5,3","7,8"
6,2018-01-07,"1,8","3,5","-0,1","3,8","3,3"
7,2018-01-08,"3,9","0,1","-1,0","8,8","1,8"
8,2018-01-09,"0,6","2,2","-2,7","4,0","3,9"
9,2018-01-10,"3,8","0,4","1,1","6,6","0,6"


#### Ahora que vemos que funciona, crearemos una funcion que haga lo explicado anteriormente

In [38]:
def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_measurements = [None]*N + [df[feature][i-N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_measurements

#### A continuacion, mediante un bucle, aplicaremos toda la funcion al dataframe principal, con estos nuevos datos podremos predecir mucho mejor el clima

In [44]:
features = ['tmed', 'prec', 'tmin', 'tmax']
for feature in features:
    if feature != 'date':
        for N in range(1, 4):
            derive_nth_day_feature(df, feature, N)